# Proyecto Capstone: Restaurantes en Toronto

## Tabla de contenidos
* [Introducción](#id1)
* [Datos](#id2)
* [Metodologia](#id3)
  * [Importar Librerias](#id3.1)
  * [Importar Data de vecindarios](#id3.2)
  * [Preprocesar Data de vecinadarios](#id3.3)
  * [Identificar Longitud y Latitud por vecindario](#id3.4)
  * [Mapa de Toronto con vecindarios](#id3.5)
* [Restaurantes por vecindarios](#id4)
  * [North York](#id4.1)
  * [Downtown Toronto](#id4.2)
  * [Queen's Park](#id4.3)
  * [Etobicoke](#id4.4)
  * [Scarborough](#id4.5)
  * [East York](#id4.6)
  * [York](#id4.7)
  * [East Toronto](#id4.8)
  * [West Toronto](#id4.9)
  * [Central Toronto](#id4.10)
* [Analisis Final](#id5)
* [Conclusiones](#id6)
    

## Introducción <a name="id1"></a>

Durante la pandemia se vivió un cambio a nivel mundial. Muchos negocios cerraron o tuvieron que adaptarse a las nuevas condiciones para sobrevivir. Lo que dio como resultado a un cambio en las zonas comerciales de Toronto y el surgimiento de nuevas oportunidades en zonas no antes exploradas.

El sector restaurantero fue unos de los más afectados, pero desde el verano pasado se ha visto un repunte de ventas casi a nivel de prepandemia. Estos factores han atraído nuevos inversionistas a este sector así que este proyecto tiene como objetivo identificar las mejores zonas de Toronto para establecer un nuevo restaurante. 




## Datos <a name="id2"></a>

Con base en la definición de nuestro problema, los factores que influirán en su decisión son:

* Numero de restaurantes por zona
* Tipo de restaurantes


Se necesitarán las siguientes fuentes de datos para generar la información requerida:

*la cantidad de restaurantes y su tipo y ubicación en cada vecindario se obtendrá usando **API de Foursquare**

## Metodologia <a name="id3"></a>

### Importar Librerias <a name="id3.1"></a>

In [2]:
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
#!pip install geopy
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors

# importar k-means desde la fase de agrupación
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # retirar el comentario de esta línea si no ha completado el laboratorio de la API de FourSquare
import folium # librería para graficar mapas 

print('Libraries imported.')

Libraries imported.


### Importar Data de vecindarios <a name="id3.2"></a>

In [3]:
Toronto_data=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', match="Toronto")
len(Toronto_data)

1

### Preprocesar Data de vecinadarios <a name="id3.3"></a>

In [4]:
df=Toronto_data[0]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       20 non-null     object
 1   1       20 non-null     object
 2   2       20 non-null     object
 3   3       20 non-null     object
 4   4       20 non-null     object
 5   5       20 non-null     object
 6   6       20 non-null     object
 7   7       20 non-null     object
 8   8       20 non-null     object
dtypes: object(9)
memory usage: 1.5+ KB


In [5]:
headers=['A','B','C','D','E','F','G','H','I']
df.columns=headers
df.head()

,A,B,C,D,E,F,G,H,I
0,M1ANot assigned,M2ANot assigned,M3ANorth York(Parkwoods),M4ANorth York(Victoria Village),M5ADowntown Toronto(Regent Park / Harbourfront),M6ANorth York(Lawrence Manor / Lawrence Heights),M7AQueen's Park(Ontario Provincial Government),M8ANot assigned,M9AEtobicoke(Islington Avenue)
1,M1BScarborough(Malvern / Rouge),M2BNot assigned,M3BNorth York(Don Mills)North,M4BEast York(Parkview Hill / Woodbine Gardens),"M5BDowntown Toronto(Garden District, Ryerson)",M6BNorth York(Glencairn),M7BNot assigned,M8BNot assigned,M9BEtobicoke(West Deane Park / Princess Garden...
2,M1CScarborough(Rouge Hill / Port Union / Highl...,M2CNot assigned,M3CNorth York(Don Mills)South(Flemingdon Park),M4CEast York(Woodbine Heights),M5CDowntown Toronto(St. James Town),M6CYork(Humewood-Cedarvale),M7CNot assigned,M8CNot assigned,M9CEtobicoke(Eringate / Bloordale Gardens / Ol...
3,M1EScarborough(Guildwood / Morningside / West ...,M2ENot assigned,M3ENot assigned,M4EEast Toronto(The Beaches),M5EDowntown Toronto(Berczy Park),M6EYork(Caledonia-Fairbanks),M7ENot assigned,M8ENot assigned,M9ENot assigned
4,M1GScarborough(Woburn),M2GNot assigned,M3GNot assigned,M4GEast York(Leaside),M5GDowntown Toronto(Central Bay Street),M6GDowntown Toronto(Christie),M7GNot assigned,M8GNot assigned,M9GNot assigned


In [6]:
t=df.stack().reset_index()
t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   level_0  180 non-null    int64 
 1   level_1  180 non-null    object
 2   0        180 non-null    object
dtypes: int64(1), object(2)
memory usage: 4.3+ KB


In [7]:
headers=['a','b','c']
t.columns=headers
t.head()

,a,b,c
0,0,A,M1ANot assigned
1,0,B,M2ANot assigned
2,0,C,M3ANorth York(Parkwoods)
3,0,D,M4ANorth York(Victoria Village)
4,0,E,M5ADowntown Toronto(Regent Park / Harbourfront)


In [8]:
t['PostalCode']=t['c'].str.slice(0,3)
t['D']=t['c'].str.slice(3,)
t.head()

,a,b,c,PostalCode,D
0,0,A,M1ANot assigned,M1A,Not assigned
1,0,B,M2ANot assigned,M2A,Not assigned
2,0,C,M3ANorth York(Parkwoods),M3A,North York(Parkwoods)
3,0,D,M4ANorth York(Victoria Village),M4A,North York(Victoria Village)
4,0,E,M5ADowntown Toronto(Regent Park / Harbourfront),M5A,Downtown Toronto(Regent Park / Harbourfront)


In [9]:
t['D']= t['D'].replace('Not assigned','',)
t.head()

,a,b,c,PostalCode,D
0,0,A,M1ANot assigned,M1A,
1,0,B,M2ANot assigned,M2A,
2,0,C,M3ANorth York(Parkwoods),M3A,North York(Parkwoods)
3,0,D,M4ANorth York(Victoria Village),M4A,North York(Victoria Village)
4,0,E,M5ADowntown Toronto(Regent Park / Harbourfront),M5A,Downtown Toronto(Regent Park / Harbourfront)


In [10]:
t['Borough1']=t['D'].str.replace('(','@(',1)
t.head()

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


,a,b,c,PostalCode,D,Borough1
0,0,A,M1ANot assigned,M1A,,
1,0,B,M2ANot assigned,M2A,,
2,0,C,M3ANorth York(Parkwoods),M3A,North York(Parkwoods),North York@(Parkwoods)
3,0,D,M4ANorth York(Victoria Village),M4A,North York(Victoria Village),North York@(Victoria Village)
4,0,E,M5ADowntown Toronto(Regent Park / Harbourfront),M5A,Downtown Toronto(Regent Park / Harbourfront),Downtown Toronto@(Regent Park / Harbourfront)


In [11]:
t[['Borough', 'Neighborhood']]=t['Borough1'].str.split('@', expand=True)
t.head()

,a,b,c,PostalCode,D,Borough1,Borough,Neighborhood
0,0,A,M1ANot assigned,M1A,,,,None
1,0,B,M2ANot assigned,M2A,,,,None
2,0,C,M3ANorth York(Parkwoods),M3A,North York(Parkwoods),North York@(Parkwoods),North York,(Parkwoods)
3,0,D,M4ANorth York(Victoria Village),M4A,North York(Victoria Village),North York@(Victoria Village),North York,(Victoria Village)
4,0,E,M5ADowntown Toronto(Regent Park / Harbourfront),M5A,Downtown Toronto(Regent Park / Harbourfront),Downtown Toronto@(Regent Park / Harbourfront),Downtown Toronto,(Regent Park / Harbourfront)


In [12]:
t= t.drop(['a','b','c','D','Borough1'],axis=1)
t.dropna(subset=['Neighborhood'], axis=0, inplace=True)
t.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,(Parkwoods)
3,M4A,North York,(Victoria Village)
4,M5A,Downtown Toronto,(Regent Park / Harbourfront)
5,M6A,North York,(Lawrence Manor / Lawrence Heights)
6,M7A,Queen's Park,(Ontario Provincial Government)


In [13]:
t['Neighborhood']=t['Neighborhood'].str.replace('(','',)
t['Neighborhood']=t['Neighborhood'].str.replace(')','',)
t['Neighborhood']=t['Neighborhood'].str.replace('/',',',)
t.head(10)

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Queen's Park,Ontario Provincial Government
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern , Rouge"
11,M3B,North York,Don MillsNorth
12,M4B,East York,"Parkview Hill , Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [14]:
h=t
h.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Queen's Park,Ontario Provincial Government


### Identificar Longitud y Latitud por vecindario <a name="id3.4"></a>

In [15]:
path='https://cocl.us/Geospatial_data'
df2 = pd.read_csv(path)
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
ll=df2
ll.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
TD = pd.merge(left=h, right=ll, left_on='PostalCode', right_on='Postal Code')
TD.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",M6A,43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,M7A,43.662301,-79.389494


In [18]:
TD =TD.drop(['Postal Code'],axis=1)
TD.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


### Mapa de Toronto con vecindarios <a name="id3.5"></a>

In [19]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="GTA_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [20]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# añadir marcadores al mapa
for lat, lng, borough, neighborhood in zip(TD['Latitude'], TD['Longitude'], TD['Borough'], TD['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [21]:
TD['Borough'].unique()

array(['North York', 'Downtown Toronto', "Queen's Park", 'Etobicoke',
       'Scarborough', 'East York', 'York', 'East Toronto', 'West Toronto',
       'East YorkEast Toronto', 'Central Toronto',
       'MississaugaCanada Post Gateway Processing Centre',
       'Downtown TorontoStn A PO Boxes25 The Esplanade',
       'EtobicokeNorthwest',
       'East TorontoBusiness reply mail Processing Centre969 Eastern'],
      dtype=object)

## Restaurantes por vecindarios <a name="id4"></a>

A continuación crearemos una muestra representativa por vecindario, solo se tomara en cuenta los restaurantes ubicados en el área central de los vecindarios

### North York <a name="id4.1"></a>

In [22]:
DToronto_data = TD[TD['Borough'] == 'North York'].reset_index(drop=True)
DToronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don MillsNorth,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073


In [23]:
address = 'North York, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.7543263, -79.44911696639593.


In [24]:
import requests

url = 'https://api.foursquare.com/v3/places/search?query=restaurant&ll=43.7543%2C-79.4491&radius=50000'

headers = {
    'Accept': 'application/json',
    'Authorization': 'fsq3yG2k5UVBywBORoDkq8cx2922ATAPkGqibnwmhcbZkWg='
            }

response = requests.get(url, headers=headers).json()

print(response)

{'results': [{'fsq_id': '5a888f7647f8767d37b92f00', 'categories': [{'id': 13145, 'name': 'Fast Food Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/fastfood_', 'suffix': '.png'}}, {'id': 13302, 'name': 'Mediterranean Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/mediterranean_', 'suffix': '.png'}}], 'chains': [], 'distance': 245, 'geocodes': {'main': {'latitude': 43.753183, 'longitude': -79.451558}, 'roof': {'latitude': 43.753183, 'longitude': -79.451558}}, 'link': '/v3/places/5a888f7647f8767d37b92f00', 'location': {'address': '832 Sheppard Ave W', 'country': 'CA', 'cross_street': 'NE corner of Sheppard & Wilmington', 'formatted_address': '832 Sheppard Ave W (NE corner of Sheppard & Wilmington), North York ON M3H 2T1', 'locality': 'North York', 'neighborhood': ['Central Toronto'], 'postcode': 'M3H 2T1', 'region': 'ON'}, 'name': 'Grill Gate', 'related_places': {}, 'timezone': 'America/Toronto'}, {'fsq_id': '55665a0b498ec5589987

In [25]:
venues_list = []
for record in response['results']:
    venues_list.append({
        'Neighborhood': 'name',
        'Neighborhood Latitude': 'lat',
        'Neighborhood Longitude': 'lng',
        'Venue': record['name'],
        'Venue Latitude': record['geocodes']['main']['latitude'],
        'Venue Longitude': record['geocodes']['main']['longitude'],
        'Venue Category': record['categories'][0]['name'],
        })

    # do stuff here to create your dataframe
    nearby_venues = pd.DataFrame(venues_list)
    print(nearby_venues)

  Neighborhood Neighborhood Latitude Neighborhood Longitude       Venue  \
0         name                   lat                    lng  Grill Gate   

   Venue Latitude  Venue Longitude        Venue Category  
0       43.753183       -79.451558  Fast Food Restaurant  
  Neighborhood Neighborhood Latitude Neighborhood Longitude  \
0         name                   lat                    lng   
1         name                   lat                    lng   

                      Venue  Venue Latitude  Venue Longitude  \
0                Grill Gate       43.753183       -79.451558   
1  Kinka Izakaya North York       43.760365       -79.410068   

         Venue Category  
0  Fast Food Restaurant  
1   Japanese Restaurant  
  Neighborhood Neighborhood Latitude Neighborhood Longitude  \
0         name                   lat                    lng   
1         name                   lat                    lng   
2         name                   lat                    lng   

                 

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=10000):
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        for record in response['results']:
            venues_list.append({
                'Neighborhood': 'name',
                'Neighborhood Latitude': 'lat',
                'Neighborhood Longitude': 'lng',
                'Venue': record['name'],
                'Venue Latitude': record['geocodes']['main']['latitude'],
                'Venue Longitude': record['geocodes']['main']['longitude'],
                'Venue Category': record['categories'][0]['name'],
                })

            # do stuff here to create your dataframe
            nearby_venues = pd.DataFrame(venues_list)
            print(nearby_venues)

In [27]:
nearby_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,name,lat,lng,Grill Gate,43.753183,-79.451558,Fast Food Restaurant
1,name,lat,lng,Kinka Izakaya North York,43.760365,-79.410068,Japanese Restaurant
2,name,lat,lng,Bobbette & Belle,43.731388,-79.403655,Bakery
3,name,lat,lng,Sababa Restaurant,43.795348,-79.433724,Mediterranean Restaurant
4,name,lat,lng,Rustic Bakery,43.715504,-79.490353,Bakery


In [28]:
NY= nearby_venues
NY.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,name,lat,lng,Grill Gate,43.753183,-79.451558,Fast Food Restaurant
1,name,lat,lng,Kinka Izakaya North York,43.760365,-79.410068,Japanese Restaurant
2,name,lat,lng,Bobbette & Belle,43.731388,-79.403655,Bakery
3,name,lat,lng,Sababa Restaurant,43.795348,-79.433724,Mediterranean Restaurant
4,name,lat,lng,Rustic Bakery,43.715504,-79.490353,Bakery


In [29]:
NY['Nh']= 'North York'
NY['Nh Lat']= '43.7543'
NY['Nh Long']= '-79.44911'
NY= NY.drop(['Neighborhood','Neighborhood Latitude','Neighborhood Longitude'],axis=1)
NY.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Nh,Nh Lat,Nh Long
0,Grill Gate,43.753183,-79.451558,Fast Food Restaurant,North York,43.7543,-79.44911
1,Kinka Izakaya North York,43.760365,-79.410068,Japanese Restaurant,North York,43.7543,-79.44911
2,Bobbette & Belle,43.731388,-79.403655,Bakery,North York,43.7543,-79.44911
3,Sababa Restaurant,43.795348,-79.433724,Mediterranean Restaurant,North York,43.7543,-79.44911
4,Rustic Bakery,43.715504,-79.490353,Bakery,North York,43.7543,-79.44911


In [30]:
NY.rename(columns={'Nh':'Neighborhood'}, inplace=True)
NY.rename(columns={'Nh Lat':'Neighborhood Latitude'}, inplace=True)
NY.rename(columns={'Nh Long':'Neighborhood Longitude'}, inplace=True)
NY.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Neighborhood,Neighborhood Latitude,Neighborhood Longitude
0,Grill Gate,43.753183,-79.451558,Fast Food Restaurant,North York,43.7543,-79.44911
1,Kinka Izakaya North York,43.760365,-79.410068,Japanese Restaurant,North York,43.7543,-79.44911
2,Bobbette & Belle,43.731388,-79.403655,Bakery,North York,43.7543,-79.44911
3,Sababa Restaurant,43.795348,-79.433724,Mediterranean Restaurant,North York,43.7543,-79.44911
4,Rustic Bakery,43.715504,-79.490353,Bakery,North York,43.7543,-79.44911


In [31]:
map_NY = folium.Map(location=[latitude, longitude], zoom_start=12)

# añadir marcadores al mapa
for lat, lng, borough in zip(NY['Venue Latitude'], NY['Venue Longitude'], NY['Venue']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_NY)  
    
map_NY

### Downtown Toronto <a name="id4.2"></a>

In [32]:
DToronto_data = TD[TD['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
DToronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [33]:
address = 'Downtown Toronto, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6563221, -79.3809161.


In [34]:
import requests

url = 'https://api.foursquare.com/v3/places/search?query=restaurant&ll=43.6563%2C-79.3809&radius=50000'

headers = {
    'Accept': 'application/json',
    'Authorization': 'fsq3yG2k5UVBywBORoDkq8cx2922ATAPkGqibnwmhcbZkWg='
            }

response = requests.get(url, headers=headers).json()

print(response)

{'results': [{'fsq_id': '51755dc7498ece19b7261641', 'categories': [{'id': 13073, 'name': 'Australian Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/australian_', 'suffix': '.png'}}, {'id': 13334, 'name': 'Sandwich Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/deli_', 'suffix': '.png'}}, {'id': 13379, 'name': 'Vietnamese Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/vietnamese_', 'suffix': '.png'}}], 'chains': [], 'distance': 343, 'geocodes': {'main': {'latitude': 43.659224, 'longitude': -79.381994}, 'roof': {'latitude': 43.659224, 'longitude': -79.381994}}, 'link': '/v3/places/51755dc7498ece19b7261641', 'location': {'address': '399 Yonge St', 'country': 'CA', 'cross_street': 'Gerrard St.', 'formatted_address': '399 Yonge St (Gerrard St.), Toronto ON M5B 1S9', 'locality': 'Toronto', 'neighborhood': ['Church and Wellesley'], 'postcode': 'M5B 1S9', 'region': 'ON'}, 'name': 'Banh Mi Boys', 'related_

In [35]:
venues_list = []
for record in response['results']:
    venues_list.append({
        'Neighborhood': 'name',
        'Neighborhood Latitude': 'lat',
        'Neighborhood Longitude': 'lng',
        'Venue': record['name'],
        'Venue Latitude': record['geocodes']['main']['latitude'],
        'Venue Longitude': record['geocodes']['main']['longitude'],
        'Venue Category': record['categories'][0]['name'],
        })

    # do stuff here to create your dataframe
    nearby_venues = pd.DataFrame(venues_list)
    print(nearby_venues)

  Neighborhood Neighborhood Latitude Neighborhood Longitude         Venue  \
0         name                   lat                    lng  Banh Mi Boys   

   Venue Latitude  Venue Longitude         Venue Category  
0       43.659224       -79.381994  Australian Restaurant  
  Neighborhood Neighborhood Latitude Neighborhood Longitude          Venue  \
0         name                   lat                    lng   Banh Mi Boys   
1         name                   lat                    lng  Page One Cafe   

   Venue Latitude  Venue Longitude         Venue Category  
0       43.659224       -79.381994  Australian Restaurant  
1       43.657465       -79.376056                   Café  
  Neighborhood Neighborhood Latitude Neighborhood Longitude             Venue  \
0         name                   lat                    lng      Banh Mi Boys   
1         name                   lat                    lng     Page One Cafe   
2         name                   lat                    lng  Richmo

In [36]:
def getNearbyVenues(names, latitudes, longitudes, radius=10000):
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        for record in response['results']:
            venues_list.append({
                'Neighborhood': 'name',
                'Neighborhood Latitude': 'lat',
                'Neighborhood Longitude': 'lng',
                'Venue': record['name'],
                'Venue Latitude': record['geocodes']['main']['latitude'],
                'Venue Longitude': record['geocodes']['main']['longitude'],
                'Venue Category': record['categories'][0]['name'],
                })

            # do stuff here to create your dataframe
            nearby_venues = pd.DataFrame(venues_list)
            print(nearby_venues)

In [37]:
nearby_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,name,lat,lng,Banh Mi Boys,43.659224,-79.381994,Australian Restaurant
1,name,lat,lng,Page One Cafe,43.657465,-79.376056,Café
2,name,lat,lng,Richmond Station,43.651510,-79.379223,American Restaurant
3,name,lat,lng,The Queen and Beaver Public House,43.657393,-79.383245,Bar
4,name,lat,lng,Blaze Pizza,43.656595,-79.380313,Pizzeria


In [38]:
DT= nearby_venues
DT.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,name,lat,lng,Banh Mi Boys,43.659224,-79.381994,Australian Restaurant
1,name,lat,lng,Page One Cafe,43.657465,-79.376056,Café
2,name,lat,lng,Richmond Station,43.651510,-79.379223,American Restaurant
3,name,lat,lng,The Queen and Beaver Public House,43.657393,-79.383245,Bar
4,name,lat,lng,Blaze Pizza,43.656595,-79.380313,Pizzeria


In [39]:
DT['Nh']= 'Downtown Toronto'
DT['Nh Lat']= '43.6563'
DT['Nh Long']= '-79.3809'
DT= DT.drop(['Neighborhood','Neighborhood Latitude','Neighborhood Longitude'],axis=1)
DT.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Nh,Nh Lat,Nh Long
0,Banh Mi Boys,43.659224,-79.381994,Australian Restaurant,Downtown Toronto,43.6563,-79.3809
1,Page One Cafe,43.657465,-79.376056,Café,Downtown Toronto,43.6563,-79.3809
2,Richmond Station,43.651510,-79.379223,American Restaurant,Downtown Toronto,43.6563,-79.3809
3,The Queen and Beaver Public House,43.657393,-79.383245,Bar,Downtown Toronto,43.6563,-79.3809
4,Blaze Pizza,43.656595,-79.380313,Pizzeria,Downtown Toronto,43.6563,-79.3809


In [40]:
DT.rename(columns={'Nh':'Neighborhood'}, inplace=True)
DT.rename(columns={'Nh Lat':'Neighborhood Latitude'}, inplace=True)
DT.rename(columns={'Nh Long':'Neighborhood Longitude'}, inplace=True)
DT.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Neighborhood,Neighborhood Latitude,Neighborhood Longitude
0,Banh Mi Boys,43.659224,-79.381994,Australian Restaurant,Downtown Toronto,43.6563,-79.3809
1,Page One Cafe,43.657465,-79.376056,Café,Downtown Toronto,43.6563,-79.3809
2,Richmond Station,43.651510,-79.379223,American Restaurant,Downtown Toronto,43.6563,-79.3809
3,The Queen and Beaver Public House,43.657393,-79.383245,Bar,Downtown Toronto,43.6563,-79.3809
4,Blaze Pizza,43.656595,-79.380313,Pizzeria,Downtown Toronto,43.6563,-79.3809


In [41]:
map_DT = folium.Map(location=[latitude, longitude], zoom_start=15)

# añadir marcadores al mapa
for lat, lng, borough in zip(DT['Venue Latitude'], DT['Venue Longitude'], DT['Venue']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_DT)  
    
map_DT

In [153]:
DT

,Venue,Venue Latitude,Venue Longitude,Venue Category,Neighborhood,Neighborhood Latitude,Neighborhood Longitude
0,Banh Mi Boys,43.659224,-79.381994,Australian Restaurant,Downtown Toronto,43.6563,-79.3809
1,Page One Cafe,43.657465,-79.376056,Café,Downtown Toronto,43.6563,-79.3809
2,Richmond Station,43.651510,-79.379223,American Restaurant,Downtown Toronto,43.6563,-79.3809
3,The Queen and Beaver Public House,43.657393,-79.383245,Bar,Downtown Toronto,43.6563,-79.3809
4,Blaze Pizza,43.656595,-79.380313,Pizzeria,Downtown Toronto,43.6563,-79.3809
5,Gyu-Kaku BBQ,43.651499,-79.374733,Japanese Restaurant,Downtown Toronto,43.6563,-79.3809
6,Mystic Muffin,43.652443,-79.372546,Café,Downtown Toronto,43.6563,-79.3809
7,The Keg Steakhouse + Bar,43.649868,-79.384058,Restaurant,Downtown Toronto,43.6563,-79.3809
8,Byblos,43.647712,-79.388249,Mediterranean Restaurant,Downtown Toronto,43.6563,-79.3809
9,Rosalinda,43.650207,-79.384813,Vegan and Vegetarian Restaurant,Downtown Toronto,43.6563,-79.3809


### Queen's Park <a name="id4.3"></a>

In [42]:
DToronto_data = TD[TD["Borough"] == "Queen's Park"].reset_index(drop=True)
DToronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [43]:
address = "Queen's Park, CA"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The geograpical coordinate of Toronto are {}, {}.".format(latitude, longitude))

The geograpical coordinate of Toronto are 43.66450665, -79.39235972390199.


In [44]:
import requests

url = 'https://api.foursquare.com/v3/places/search?query=restaurant&ll=43.6645%2C-79.3923&radius=50000'

headers = {
    'Accept': 'application/json',
    'Authorization': 'fsq3yG2k5UVBywBORoDkq8cx2922ATAPkGqibnwmhcbZkWg='
            }

response = requests.get(url, headers=headers).json()

print(response)

{'results': [{'fsq_id': '527d450111d25050de4ea0d8', 'categories': [{'id': 13065, 'name': 'Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_', 'suffix': '.png'}}], 'chains': [], 'distance': 959, 'geocodes': {'main': {'latitude': 43.662949, 'longitude': -79.404143}, 'roof': {'latitude': 43.662949, 'longitude': -79.404143}}, 'link': '/v3/places/527d450111d25050de4ea0d8', 'location': {'address': '196 Robert St', 'country': 'CA', 'cross_street': '', 'formatted_address': '196 Robert St, Toronto ON M5S 2K7', 'locality': 'Toronto', 'neighborhood': ['Downtown Toronto'], 'postcode': 'M5S 2K7', 'region': 'ON'}, 'name': 'Rasa', 'related_places': {}, 'timezone': 'America/Toronto'}, {'fsq_id': '5bd2379cbcbf7a0039a2d7b9', 'categories': [{'id': 13003, 'name': 'Bar', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_', 'suffix': '.png'}}, {'id': 13354, 'name': 'Theme Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/d

In [45]:
venues_list = []
for record in response['results']:
    venues_list.append({
        'Neighborhood': 'name',
        'Neighborhood Latitude': 'lat',
        'Neighborhood Longitude': 'lng',
        'Venue': record['name'],
        'Venue Latitude': record['geocodes']['main']['latitude'],
        'Venue Longitude': record['geocodes']['main']['longitude'],
        'Venue Category': record['categories'][0]['name'],
        })

    # do stuff here to create your dataframe
    nearby_venues = pd.DataFrame(venues_list)
    print(nearby_venues)

  Neighborhood Neighborhood Latitude Neighborhood Longitude Venue  \
0         name                   lat                    lng  Rasa   

   Venue Latitude  Venue Longitude Venue Category  
0       43.662949       -79.404143     Restaurant  
  Neighborhood Neighborhood Latitude Neighborhood Longitude             Venue  \
0         name                   lat                    lng              Rasa   
1         name                   lat                    lng  Storm Crow Manor   

   Venue Latitude  Venue Longitude Venue Category  
0       43.662949       -79.404143     Restaurant  
1       43.666761       -79.381790            Bar  
  Neighborhood Neighborhood Latitude Neighborhood Longitude             Venue  \
0         name                   lat                    lng              Rasa   
1         name                   lat                    lng  Storm Crow Manor   
2         name                   lat                    lng      Banh Mi Boys   

   Venue Latitude  Venue Longitu

In [46]:
def getNearbyVenues(names, latitudes, longitudes, radius=10000):
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        for record in response['results']:
            venues_list.append({
                'Neighborhood': 'name',
                'Neighborhood Latitude': 'lat',
                'Neighborhood Longitude': 'lng',
                'Venue': record['name'],
                'Venue Latitude': record['geocodes']['main']['latitude'],
                'Venue Longitude': record['geocodes']['main']['longitude'],
                'Venue Category': record['categories'][0]['name'],
                })

            # do stuff here to create your dataframe
            nearby_venues = pd.DataFrame(venues_list)
            print(nearby_venues)

In [47]:
nearby_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,name,lat,lng,Rasa,43.662949,-79.404143,Restaurant
1,name,lat,lng,Storm Crow Manor,43.666761,-79.381790,Bar
2,name,lat,lng,Banh Mi Boys,43.659224,-79.381994,Australian Restaurant
3,name,lat,lng,Black Camel,43.677100,-79.389423,BBQ Joint
4,name,lat,lng,The Queen and Beaver Public House,43.657393,-79.383245,Bar


In [48]:
QP= nearby_venues
QP.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,name,lat,lng,Rasa,43.662949,-79.404143,Restaurant
1,name,lat,lng,Storm Crow Manor,43.666761,-79.381790,Bar
2,name,lat,lng,Banh Mi Boys,43.659224,-79.381994,Australian Restaurant
3,name,lat,lng,Black Camel,43.677100,-79.389423,BBQ Joint
4,name,lat,lng,The Queen and Beaver Public House,43.657393,-79.383245,Bar


In [49]:
QP["Nh"]= "Queen's Park"
QP["Nh Lat"]= "43.6645"
QP["Nh Long"]= "-79.3923"
QP= QP.drop(["Neighborhood","Neighborhood Latitude","Neighborhood Longitude"],axis=1)
QP.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Nh,Nh Lat,Nh Long
0,Rasa,43.662949,-79.404143,Restaurant,Queen's Park,43.6645,-79.3923
1,Storm Crow Manor,43.666761,-79.381790,Bar,Queen's Park,43.6645,-79.3923
2,Banh Mi Boys,43.659224,-79.381994,Australian Restaurant,Queen's Park,43.6645,-79.3923
3,Black Camel,43.677100,-79.389423,BBQ Joint,Queen's Park,43.6645,-79.3923
4,The Queen and Beaver Public House,43.657393,-79.383245,Bar,Queen's Park,43.6645,-79.3923


In [50]:
QP.rename(columns={'Nh':'Neighborhood'}, inplace=True)
QP.rename(columns={'Nh Lat':'Neighborhood Latitude'}, inplace=True)
QP.rename(columns={'Nh Long':'Neighborhood Longitude'}, inplace=True)
QP.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Neighborhood,Neighborhood Latitude,Neighborhood Longitude
0,Rasa,43.662949,-79.404143,Restaurant,Queen's Park,43.6645,-79.3923
1,Storm Crow Manor,43.666761,-79.381790,Bar,Queen's Park,43.6645,-79.3923
2,Banh Mi Boys,43.659224,-79.381994,Australian Restaurant,Queen's Park,43.6645,-79.3923
3,Black Camel,43.677100,-79.389423,BBQ Joint,Queen's Park,43.6645,-79.3923
4,The Queen and Beaver Public House,43.657393,-79.383245,Bar,Queen's Park,43.6645,-79.3923


In [51]:
map_QP = folium.Map(location=[latitude, longitude], zoom_start=14)

# añadir marcadores al mapa
for lat, lng, borough in zip(QP['Venue Latitude'], QP['Venue Longitude'], QP['Venue']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_QP)  
    
map_QP

### Etobicoke <a name="id4.4"></a>

In [52]:
DToronto_data = TD[TD['Borough'] == 'Etobicoke'].reset_index(drop=True)
DToronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
1,M9B,Etobicoke,"West Deane Park , Princess Gardens , Martin Gr...",43.650943,-79.554724
2,M9C,Etobicoke,"Eringate , Bloordale Gardens , Old Burnhamthor...",43.643515,-79.577201
3,M9P,Etobicoke,Westmount,43.696319,-79.532242
4,M9R,Etobicoke,"Kingsview Village , St. Phillips , Martin Grov...",43.688905,-79.554724


In [53]:
address = 'Etobicoke, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6435559, -79.5656326.


In [54]:
import requests

url = 'https://api.foursquare.com/v3/places/search?query=restaurant&ll=43.6435%2C-79.5656&radius=50000'

headers = {
    'Accept': 'application/json',
    'Authorization': 'fsq3yG2k5UVBywBORoDkq8cx2922ATAPkGqibnwmhcbZkWg='
            }

response = requests.get(url, headers=headers).json()

print(response)

{'results': [{'fsq_id': '4af4709bf964a52079f221e3', 'categories': [{'id': 13236, 'name': 'Italian Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_', 'suffix': '.png'}}, {'id': 13334, 'name': 'Sandwich Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/deli_', 'suffix': '.png'}}], 'chains': [], 'distance': 2941, 'geocodes': {'main': {'latitude': 43.663569, 'longitude': -79.588923}, 'roof': {'latitude': 43.663569, 'longitude': -79.588923}}, 'link': '/v3/places/4af4709bf964a52079f221e3', 'location': {'address': '5399 Eglinton Ave W', 'address_extended': 'Unit 104', 'country': 'CA', 'cross_street': 'btw Renforth & Commerce', 'formatted_address': '5399 Eglinton Ave W (btw Renforth & Commerce), Toronto ON M9C 5K6', 'locality': 'Toronto', 'neighborhood': ['Richview'], 'postcode': 'M9C 5K6', 'region': 'ON'}, 'name': 'Porta Via', 'related_places': {}, 'timezone': 'America/Toronto'}, {'fsq_id': '56c8dbe9498e7033010ac0e5', 'categories

In [55]:
venues_list = []
for record in response['results']:
    venues_list.append({
        'Neighborhood': 'name',
        'Neighborhood Latitude': 'lat',
        'Neighborhood Longitude': 'lng',
        'Venue': record['name'],
        'Venue Latitude': record['geocodes']['main']['latitude'],
        'Venue Longitude': record['geocodes']['main']['longitude'],
        'Venue Category': record['categories'][0]['name'],
        })

    # do stuff here to create your dataframe
    nearby_venues = pd.DataFrame(venues_list)
    print(nearby_venues)

  Neighborhood Neighborhood Latitude Neighborhood Longitude      Venue  \
0         name                   lat                    lng  Porta Via   

   Venue Latitude  Venue Longitude      Venue Category  
0       43.663569       -79.588923  Italian Restaurant  
  Neighborhood Neighborhood Latitude Neighborhood Longitude         Venue  \
0         name                   lat                    lng     Porta Via   
1         name                   lat                    lng  JOEY Sherway   

   Venue Latitude  Venue Longitude           Venue Category  
0       43.663569       -79.588923       Italian Restaurant  
1       43.612984       -79.556447  New American Restaurant  
  Neighborhood Neighborhood Latitude Neighborhood Longitude  \
0         name                   lat                    lng   
1         name                   lat                    lng   
2         name                   lat                    lng   

                      Venue  Venue Latitude  Venue Longitude  \
0 

In [56]:
def getNearbyVenues(names, latitudes, longitudes, radius=10000):
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        for record in response['results']:
            venues_list.append({
                'Neighborhood': 'name',
                'Neighborhood Latitude': 'lat',
                'Neighborhood Longitude': 'lng',
                'Venue': record['name'],
                'Venue Latitude': record['geocodes']['main']['latitude'],
                'Venue Longitude': record['geocodes']['main']['longitude'],
                'Venue Category': record['categories'][0]['name'],
                })

            # do stuff here to create your dataframe
            nearby_venues = pd.DataFrame(venues_list)
            print(nearby_venues)

In [57]:
nearby_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,name,lat,lng,Porta Via,43.663569,-79.588923,Italian Restaurant
1,name,lat,lng,JOEY Sherway,43.612984,-79.556447,New American Restaurant
2,name,lat,lng,The Keg Steakhouse + Bar,43.611540,-79.557006,Steakhouse
3,name,lat,lng,Sunny Morning Breakfast & Lunch,43.618391,-79.544059,American Restaurant
4,name,lat,lng,Woody's Burgers,43.592343,-79.541925,BBQ Joint


In [58]:
ETO= nearby_venues
ETO.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,name,lat,lng,Porta Via,43.663569,-79.588923,Italian Restaurant
1,name,lat,lng,JOEY Sherway,43.612984,-79.556447,New American Restaurant
2,name,lat,lng,The Keg Steakhouse + Bar,43.611540,-79.557006,Steakhouse
3,name,lat,lng,Sunny Morning Breakfast & Lunch,43.618391,-79.544059,American Restaurant
4,name,lat,lng,Woody's Burgers,43.592343,-79.541925,BBQ Joint


In [59]:
ETO['Nh']= 'Etobicoke'
ETO['Nh Lat']= '43.6435'
ETO['Nh Long']= '-79.5656'
ETO= ETO.drop(['Neighborhood','Neighborhood Latitude','Neighborhood Longitude'],axis=1)
ETO.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Nh,Nh Lat,Nh Long
0,Porta Via,43.663569,-79.588923,Italian Restaurant,Etobicoke,43.6435,-79.5656
1,JOEY Sherway,43.612984,-79.556447,New American Restaurant,Etobicoke,43.6435,-79.5656
2,The Keg Steakhouse + Bar,43.611540,-79.557006,Steakhouse,Etobicoke,43.6435,-79.5656
3,Sunny Morning Breakfast & Lunch,43.618391,-79.544059,American Restaurant,Etobicoke,43.6435,-79.5656
4,Woody's Burgers,43.592343,-79.541925,BBQ Joint,Etobicoke,43.6435,-79.5656


In [60]:
ETO.rename(columns={'Nh':'Neighborhood'}, inplace=True)
ETO.rename(columns={'Nh Lat':'Neighborhood Latitude'}, inplace=True)
ETO.rename(columns={'Nh Long':'Neighborhood Longitude'}, inplace=True)
ETO.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Neighborhood,Neighborhood Latitude,Neighborhood Longitude
0,Porta Via,43.663569,-79.588923,Italian Restaurant,Etobicoke,43.6435,-79.5656
1,JOEY Sherway,43.612984,-79.556447,New American Restaurant,Etobicoke,43.6435,-79.5656
2,The Keg Steakhouse + Bar,43.611540,-79.557006,Steakhouse,Etobicoke,43.6435,-79.5656
3,Sunny Morning Breakfast & Lunch,43.618391,-79.544059,American Restaurant,Etobicoke,43.6435,-79.5656
4,Woody's Burgers,43.592343,-79.541925,BBQ Joint,Etobicoke,43.6435,-79.5656


In [61]:
map_ETO = folium.Map(location=[latitude, longitude], zoom_start=15)

# añadir marcadores al mapa
for lat, lng, borough in zip(ETO['Venue Latitude'], ETO['Venue Longitude'], ETO['Venue']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ETO)  
    
map_ETO

In [154]:
ETO

,Venue,Venue Latitude,Venue Longitude,Venue Category,Neighborhood,Neighborhood Latitude,Neighborhood Longitude
0,Porta Via,43.663569,-79.588923,Italian Restaurant,Etobicoke,43.6435,-79.5656
1,JOEY Sherway,43.612984,-79.556447,New American Restaurant,Etobicoke,43.6435,-79.5656
2,The Keg Steakhouse + Bar,43.611540,-79.557006,Steakhouse,Etobicoke,43.6435,-79.5656
3,Sunny Morning Breakfast & Lunch,43.618391,-79.544059,American Restaurant,Etobicoke,43.6435,-79.5656
4,Woody's Burgers,43.592343,-79.541925,BBQ Joint,Etobicoke,43.6435,-79.5656
5,The Keg Steakhouse + Bar,43.686405,-79.597530,Steakhouse,Etobicoke,43.6435,-79.5656
6,Huevos Gourmet,43.601284,-79.503634,Mexican Restaurant,Etobicoke,43.6435,-79.5656
7,Cheese Boutique,43.638456,-79.475109,Deli,Etobicoke,43.6435,-79.5656
8,The Good Neighbour,43.662602,-79.471193,Café,Etobicoke,43.6435,-79.5656
9,Chubby's Fish N' Chips,43.715295,-79.555680,Fish and Chips Shop,Etobicoke,43.6435,-79.5656


### Scarborough <a name="id4.5"></a>

In [62]:
DToronto_data = TD[TD['Borough'] == 'Scarborough'].reset_index(drop=True)
DToronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [63]:
address = 'Scarborough, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.7729744, -79.2576479.


In [64]:
import requests

url = 'https://api.foursquare.com/v3/places/search?query=restaurant&ll=43.7729%2C-79.2576&radius=50000'

headers = {
    'Accept': 'application/json',
    'Authorization': 'fsq3yG2k5UVBywBORoDkq8cx2922ATAPkGqibnwmhcbZkWg='
            }

response = requests.get(url, headers=headers).json()

print(response)

{'results': [{'fsq_id': '4c5c9bee94fd0f47cb30c945', 'categories': [{'id': 13097, 'name': 'Caribbean Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/caribbean_', 'suffix': '.png'}}], 'chains': [], 'distance': 1600, 'geocodes': {'main': {'latitude': 43.78698, 'longitude': -79.258242}, 'roof': {'latitude': 43.78698, 'longitude': -79.258242}}, 'link': '/v3/places/4c5c9bee94fd0f47cb30c945', 'location': {'address': '351 Pitfield Rd', 'country': 'CA', 'cross_street': 'McCowan Rd', 'formatted_address': '351 Pitfield Rd (McCowan Rd), Scarborough ON M1S 3E5', 'locality': 'Scarborough', 'neighborhood': ['Scarborough'], 'postcode': 'M1S 3E5', 'region': 'ON'}, 'name': 'The Roti Hut', 'related_places': {'parent': {'fsq_id': '5fdfc3a988687741416469fd', 'name': 'Pitfield Plaza'}}, 'timezone': 'America/Toronto'}, {'fsq_id': '4b340050f964a520b42325e3', 'categories': [{'id': 13383, 'name': 'Steakhouse', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/steakhous

In [65]:
venues_list = []
for record in response['results']:
    venues_list.append({
        'Neighborhood': 'name',
        'Neighborhood Latitude': 'lat',
        'Neighborhood Longitude': 'lng',
        'Venue': record['name'],
        'Venue Latitude': record['geocodes']['main']['latitude'],
        'Venue Longitude': record['geocodes']['main']['longitude'],
        'Venue Category': record['categories'][0]['name'],
        })

    # do stuff here to create your dataframe
    nearby_venues = pd.DataFrame(venues_list)
    print(nearby_venues)

  Neighborhood Neighborhood Latitude Neighborhood Longitude         Venue  \
0         name                   lat                    lng  The Roti Hut   

   Venue Latitude  Venue Longitude        Venue Category  
0        43.78698       -79.258242  Caribbean Restaurant  
  Neighborhood Neighborhood Latitude Neighborhood Longitude  \
0         name                   lat                    lng   
1         name                   lat                    lng   

                      Venue  Venue Latitude  Venue Longitude  \
0              The Roti Hut       43.786980       -79.258242   
1  The Keg Steakhouse + Bar       43.782615       -79.235103   

         Venue Category  
0  Caribbean Restaurant  
1            Steakhouse  
  Neighborhood Neighborhood Latitude Neighborhood Longitude  \
0         name                   lat                    lng   
1         name                   lat                    lng   
2         name                   lat                    lng   

             

In [66]:
def getNearbyVenues(names, latitudes, longitudes, radius=10000):
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        for record in response['results']:
            venues_list.append({
                'Neighborhood': 'name',
                'Neighborhood Latitude': 'lat',
                'Neighborhood Longitude': 'lng',
                'Venue': record['name'],
                'Venue Latitude': record['geocodes']['main']['latitude'],
                'Venue Longitude': record['geocodes']['main']['longitude'],
                'Venue Category': record['categories'][0]['name'],
                })

            # do stuff here to create your dataframe
            nearby_venues = pd.DataFrame(venues_list)
            print(nearby_venues)

In [67]:
nearby_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,name,lat,lng,The Roti Hut,43.786980,-79.258242,Caribbean Restaurant
1,name,lat,lng,The Keg Steakhouse + Bar,43.782615,-79.235103,Steakhouse
2,name,lat,lng,Pho Metro,43.745302,-79.294156,Asian Restaurant
3,name,lat,lng,Fishman Lobster Clubhouse Restaurant 魚樂軒,43.801927,-79.295243,Chinese Restaurant
4,name,lat,lng,Fahmee Bakery & Jamaican Foods,43.810114,-79.279833,Bakery


In [68]:
SH= nearby_venues
SH.head()


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,name,lat,lng,The Roti Hut,43.786980,-79.258242,Caribbean Restaurant
1,name,lat,lng,The Keg Steakhouse + Bar,43.782615,-79.235103,Steakhouse
2,name,lat,lng,Pho Metro,43.745302,-79.294156,Asian Restaurant
3,name,lat,lng,Fishman Lobster Clubhouse Restaurant 魚樂軒,43.801927,-79.295243,Chinese Restaurant
4,name,lat,lng,Fahmee Bakery & Jamaican Foods,43.810114,-79.279833,Bakery


In [69]:
SH['Nh']= 'Scarborough'
SH['Nh Lat']= '43.7729'
SH['Nh Long']= '-79.2576'
SH= SH.drop(['Neighborhood','Neighborhood Latitude','Neighborhood Longitude'],axis=1)
SH.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Nh,Nh Lat,Nh Long
0,The Roti Hut,43.786980,-79.258242,Caribbean Restaurant,Scarborough,43.7729,-79.2576
1,The Keg Steakhouse + Bar,43.782615,-79.235103,Steakhouse,Scarborough,43.7729,-79.2576
2,Pho Metro,43.745302,-79.294156,Asian Restaurant,Scarborough,43.7729,-79.2576
3,Fishman Lobster Clubhouse Restaurant 魚樂軒,43.801927,-79.295243,Chinese Restaurant,Scarborough,43.7729,-79.2576
4,Fahmee Bakery & Jamaican Foods,43.810114,-79.279833,Bakery,Scarborough,43.7729,-79.2576


In [70]:
SH.rename(columns={'Nh':'Neighborhood'}, inplace=True)
SH.rename(columns={'Nh Lat':'Neighborhood Latitude'}, inplace=True)
SH.rename(columns={'Nh Long':'Neighborhood Longitude'}, inplace=True)
SH.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Neighborhood,Neighborhood Latitude,Neighborhood Longitude
0,The Roti Hut,43.786980,-79.258242,Caribbean Restaurant,Scarborough,43.7729,-79.2576
1,The Keg Steakhouse + Bar,43.782615,-79.235103,Steakhouse,Scarborough,43.7729,-79.2576
2,Pho Metro,43.745302,-79.294156,Asian Restaurant,Scarborough,43.7729,-79.2576
3,Fishman Lobster Clubhouse Restaurant 魚樂軒,43.801927,-79.295243,Chinese Restaurant,Scarborough,43.7729,-79.2576
4,Fahmee Bakery & Jamaican Foods,43.810114,-79.279833,Bakery,Scarborough,43.7729,-79.2576


In [71]:
map_SH = folium.Map(location=[latitude, longitude], zoom_start=14)

# añadir marcadores al mapa
for lat, lng, borough in zip(SH['Venue Latitude'], SH['Venue Longitude'], SH['Venue']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_SH)  
    
map_SH

### East York <a name="id4.6"></a>

In [72]:
DToronto_data = TD[TD['Borough'] == 'East York'].reset_index(drop=True)
DToronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937
1,M4C,East York,Woodbine Heights,43.695344,-79.318389
2,M4G,East York,Leaside,43.709060,-79.363452
3,M4H,East York,Thorncliffe Park,43.705369,-79.349372


In [73]:
address = 'East York, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.699971000000005, -79.33251996261595.


In [74]:
import requests

url = 'https://api.foursquare.com/v3/places/search?query=restaurant&ll=43.6999%2C-79.3325&radius=50000'

headers = {
    'Accept': 'application/json',
    'Authorization': 'fsq3yG2k5UVBywBORoDkq8cx2922ATAPkGqibnwmhcbZkWg='
            }

response = requests.get(url, headers=headers).json()

print(response)

{'results': [{'fsq_id': '4ad9221cf964a520671821e3', 'categories': [{'id': 13067, 'name': 'African Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/african_', 'suffix': '.png'}}, {'id': 13143, 'name': 'Ethiopian Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/ethiopian_', 'suffix': '.png'}}, {'id': 13377, 'name': 'Vegan and Vegetarian Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/vegetarian_', 'suffix': '.png'}}], 'chains': [], 'distance': 1966, 'geocodes': {'main': {'latitude': 43.682651, 'longitude': -79.327389}, 'roof': {'latitude': 43.682651, 'longitude': -79.327389}}, 'link': '/v3/places/4ad9221cf964a520671821e3', 'location': {'address': '1408 Danforth Ave', 'country': 'CA', 'cross_street': 'Monarch Park Ave.', 'formatted_address': '1408 Danforth Ave (Monarch Park Ave.), Toronto ON M4J 1M9', 'locality': 'Toronto', 'neighborhood': ['Greenwood-Coxwell'], 'postcode': 'M4J 1M9', 'region': 'ON'}, 'na

In [75]:
venues_list = []
for record in response['results']:
    venues_list.append({
        'Neighborhood': 'name',
        'Neighborhood Latitude': 'lat',
        'Neighborhood Longitude': 'lng',
        'Venue': record['name'],
        'Venue Latitude': record['geocodes']['main']['latitude'],
        'Venue Longitude': record['geocodes']['main']['longitude'],
        'Venue Category': record['categories'][0]['name'],
        })

    # do stuff here to create your dataframe
    nearby_venues = pd.DataFrame(venues_list)
    print(nearby_venues)

  Neighborhood Neighborhood Latitude Neighborhood Longitude        Venue  \
0         name                   lat                    lng  Rendez-Vous   

   Venue Latitude  Venue Longitude      Venue Category  
0       43.682651       -79.327389  African Restaurant  
  Neighborhood Neighborhood Latitude Neighborhood Longitude          Venue  \
0         name                   lat                    lng    Rendez-Vous   
1         name                   lat                    lng  Serano Bakery   

   Venue Latitude  Venue Longitude      Venue Category  
0       43.682651       -79.327389  African Restaurant  
1       43.682890       -79.346653              Bakery  
  Neighborhood Neighborhood Latitude Neighborhood Longitude          Venue  \
0         name                   lat                    lng    Rendez-Vous   
1         name                   lat                    lng  Serano Bakery   
2         name                   lat                    lng       The Wren   

   Venue Latit

In [76]:
def getNearbyVenues(names, latitudes, longitudes, radius=10000):
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        for record in response['results']:
            venues_list.append({
                'Neighborhood': 'name',
                'Neighborhood Latitude': 'lat',
                'Neighborhood Longitude': 'lng',
                'Venue': record['name'],
                'Venue Latitude': record['geocodes']['main']['latitude'],
                'Venue Longitude': record['geocodes']['main']['longitude'],
                'Venue Category': record['categories'][0]['name'],
                })

            # do stuff here to create your dataframe
            nearby_venues = pd.DataFrame(venues_list)
            print(nearby_venues)

In [77]:
nearby_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,name,lat,lng,Rendez-Vous,43.682651,-79.327389,African Restaurant
1,name,lat,lng,Serano Bakery,43.682890,-79.346653,Bakery
2,name,lat,lng,The Wren,43.682552,-79.327981,Bar
3,name,lat,lng,Red Rocket Coffee,43.682512,-79.328536,Bakery
4,name,lat,lng,Olde Yorke Fish & Chips,43.706202,-79.362095,Fish and Chips Shop


In [78]:
EY= nearby_venues
EY.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,name,lat,lng,Rendez-Vous,43.682651,-79.327389,African Restaurant
1,name,lat,lng,Serano Bakery,43.682890,-79.346653,Bakery
2,name,lat,lng,The Wren,43.682552,-79.327981,Bar
3,name,lat,lng,Red Rocket Coffee,43.682512,-79.328536,Bakery
4,name,lat,lng,Olde Yorke Fish & Chips,43.706202,-79.362095,Fish and Chips Shop


In [79]:
EY['Nh']= 'East York'
EY['Nh Lat']= '43.6999'
EY['Nh Long']= '-79.3325'
EY= EY.drop(['Neighborhood','Neighborhood Latitude','Neighborhood Longitude'],axis=1)
EY.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Nh,Nh Lat,Nh Long
0,Rendez-Vous,43.682651,-79.327389,African Restaurant,East York,43.6999,-79.3325
1,Serano Bakery,43.682890,-79.346653,Bakery,East York,43.6999,-79.3325
2,The Wren,43.682552,-79.327981,Bar,East York,43.6999,-79.3325
3,Red Rocket Coffee,43.682512,-79.328536,Bakery,East York,43.6999,-79.3325
4,Olde Yorke Fish & Chips,43.706202,-79.362095,Fish and Chips Shop,East York,43.6999,-79.3325


In [80]:
EY.rename(columns={'Nh':'Neighborhood'}, inplace=True)
EY.rename(columns={'Nh Lat':'Neighborhood Latitude'}, inplace=True)
EY.rename(columns={'Nh Long':'Neighborhood Longitude'}, inplace=True)
EY.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Neighborhood,Neighborhood Latitude,Neighborhood Longitude
0,Rendez-Vous,43.682651,-79.327389,African Restaurant,East York,43.6999,-79.3325
1,Serano Bakery,43.682890,-79.346653,Bakery,East York,43.6999,-79.3325
2,The Wren,43.682552,-79.327981,Bar,East York,43.6999,-79.3325
3,Red Rocket Coffee,43.682512,-79.328536,Bakery,East York,43.6999,-79.3325
4,Olde Yorke Fish & Chips,43.706202,-79.362095,Fish and Chips Shop,East York,43.6999,-79.3325


In [81]:
map_EY = folium.Map(location=[latitude, longitude], zoom_start=12)

# añadir marcadores al mapa
for lat, lng, borough in zip(EY['Venue Latitude'], EY['Venue Longitude'], EY['Venue']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_EY)  
    
map_EY

### York <a name="id4.7"></a>

In [82]:
DToronto_data = TD[TD['Borough'] == 'York'].reset_index(drop=True)
DToronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M6C,York,Humewood-Cedarvale,43.693781,-79.428191
1,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512
2,M6M,York,"Del Ray , Mount Dennis , Keelsdale and Silvert...",43.691116,-79.476013
3,M6N,York,"Runnymede , The Junction North",43.673185,-79.487262
4,M9N,York,Weston,43.706876,-79.518188


In [83]:
address = 'York, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6896191, -79.479188.


In [84]:
import requests

url = 'https://api.foursquare.com/v3/places/search?query=restaurant&ll=43.6896%2C-79.4791&radius=50000'

headers = {
    'Accept': 'application/json',
    'Authorization': 'fsq3yG2k5UVBywBORoDkq8cx2922ATAPkGqibnwmhcbZkWg='
            }

response = requests.get(url, headers=headers).json()

print(response)

{'results': [{'fsq_id': '5547aad8498e00f3f580ea93', 'categories': [{'id': 13276, 'name': 'Sushi Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/sushi_', 'suffix': '.png'}}], 'chains': [], 'distance': 1923, 'geocodes': {'main': {'latitude': 43.694544, 'longitude': -79.455986}, 'roof': {'latitude': 43.694544, 'longitude': -79.455986}}, 'link': '/v3/places/5547aad8498e00f3f580ea93', 'location': {'address': '2030 Eglinton Ave W', 'country': 'CA', 'cross_street': '', 'formatted_address': '2030 Eglinton Ave W, York ON M6E 2K3', 'locality': 'York', 'neighborhood': ['York'], 'postcode': 'M6E 2K3', 'region': 'ON'}, 'name': 'Sushi Zuki', 'related_places': {}, 'timezone': 'America/Toronto'}, {'fsq_id': '4b5c7ad5f964a520df3129e3', 'categories': [{'id': 13002, 'name': 'Bakery', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_', 'suffix': '.png'}}, {'id': 13039, 'name': 'Deli', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/deli_',

In [85]:
venues_list = []
for record in response['results']:
    venues_list.append({
        'Neighborhood': 'name',
        'Neighborhood Latitude': 'lat',
        'Neighborhood Longitude': 'lng',
        'Venue': record['name'],
        'Venue Latitude': record['geocodes']['main']['latitude'],
        'Venue Longitude': record['geocodes']['main']['longitude'],
        'Venue Category': record['categories'][0]['name'],
        })

    # do stuff here to create your dataframe
    nearby_venues = pd.DataFrame(venues_list)
    print(nearby_venues)

  Neighborhood Neighborhood Latitude Neighborhood Longitude       Venue  \
0         name                   lat                    lng  Sushi Zuki   

   Venue Latitude  Venue Longitude    Venue Category  
0       43.694544       -79.455986  Sushi Restaurant  
  Neighborhood Neighborhood Latitude Neighborhood Longitude            Venue  \
0         name                   lat                    lng       Sushi Zuki   
1         name                   lat                    lng  Tre Mari Bakery   

   Venue Latitude  Venue Longitude    Venue Category  
0       43.694544       -79.455986  Sushi Restaurant  
1       43.676559       -79.448509            Bakery  
  Neighborhood Neighborhood Latitude Neighborhood Longitude            Venue  \
0         name                   lat                    lng       Sushi Zuki   
1         name                   lat                    lng  Tre Mari Bakery   
2         name                   lat                    lng             Nodo   

   Venue Lat

In [86]:
def getNearbyVenues(names, latitudes, longitudes, radius=10000):
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        for record in response['results']:
            venues_list.append({
                'Neighborhood': 'name',
                'Neighborhood Latitude': 'lat',
                'Neighborhood Longitude': 'lng',
                'Venue': record['name'],
                'Venue Latitude': record['geocodes']['main']['latitude'],
                'Venue Longitude': record['geocodes']['main']['longitude'],
                'Venue Category': record['categories'][0]['name'],
                })

            # do stuff here to create your dataframe
            nearby_venues = pd.DataFrame(venues_list)
            print(nearby_venues)

In [87]:
nearby_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,name,lat,lng,Sushi Zuki,43.694544,-79.455986,Sushi Restaurant
1,name,lat,lng,Tre Mari Bakery,43.676559,-79.448509,Bakery
2,name,lat,lng,Nodo,43.665327,-79.465412,Italian Restaurant
3,name,lat,lng,Honest Weight,43.665390,-79.461494,Seafood Restaurant
4,name,lat,lng,The Good Neighbour,43.662602,-79.471193,Café


In [88]:
YK= nearby_venues
YK.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,name,lat,lng,Sushi Zuki,43.694544,-79.455986,Sushi Restaurant
1,name,lat,lng,Tre Mari Bakery,43.676559,-79.448509,Bakery
2,name,lat,lng,Nodo,43.665327,-79.465412,Italian Restaurant
3,name,lat,lng,Honest Weight,43.665390,-79.461494,Seafood Restaurant
4,name,lat,lng,The Good Neighbour,43.662602,-79.471193,Café


In [89]:
YK['Nh']= 'York'
YK['Nh Lat']= '43.6896'
YK['Nh Long']= '-79.4791'
YK= YK.drop(['Neighborhood','Neighborhood Latitude','Neighborhood Longitude'],axis=1)
YK.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Nh,Nh Lat,Nh Long
0,Sushi Zuki,43.694544,-79.455986,Sushi Restaurant,York,43.6896,-79.4791
1,Tre Mari Bakery,43.676559,-79.448509,Bakery,York,43.6896,-79.4791
2,Nodo,43.665327,-79.465412,Italian Restaurant,York,43.6896,-79.4791
3,Honest Weight,43.665390,-79.461494,Seafood Restaurant,York,43.6896,-79.4791
4,The Good Neighbour,43.662602,-79.471193,Café,York,43.6896,-79.4791


In [90]:
YK.rename(columns={'Nh':'Neighborhood'}, inplace=True)
YK.rename(columns={'Nh Lat':'Neighborhood Latitude'}, inplace=True)
YK.rename(columns={'Nh Long':'Neighborhood Longitude'}, inplace=True)
YK.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Neighborhood,Neighborhood Latitude,Neighborhood Longitude
0,Sushi Zuki,43.694544,-79.455986,Sushi Restaurant,York,43.6896,-79.4791
1,Tre Mari Bakery,43.676559,-79.448509,Bakery,York,43.6896,-79.4791
2,Nodo,43.665327,-79.465412,Italian Restaurant,York,43.6896,-79.4791
3,Honest Weight,43.665390,-79.461494,Seafood Restaurant,York,43.6896,-79.4791
4,The Good Neighbour,43.662602,-79.471193,Café,York,43.6896,-79.4791


In [91]:
map_YK = folium.Map(location=[latitude, longitude], zoom_start=12)

# añadir marcadores al mapa
for lat, lng, borough in zip(YK['Venue Latitude'], YK['Venue Longitude'], YK['Venue']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_YK)  
    
map_YK

In [151]:
YK

,Venue,Venue Latitude,Venue Longitude,Venue Category,Neighborhood,Neighborhood Latitude,Neighborhood Longitude
0,Sushi Zuki,43.694544,-79.455986,Sushi Restaurant,York,43.6896,-79.4791
1,Tre Mari Bakery,43.676559,-79.448509,Bakery,York,43.6896,-79.4791
2,Nodo,43.665327,-79.465412,Italian Restaurant,York,43.6896,-79.4791
3,Honest Weight,43.665390,-79.461494,Seafood Restaurant,York,43.6896,-79.4791
4,The Good Neighbour,43.662602,-79.471193,Café,York,43.6896,-79.4791
5,Rustic Bakery,43.715504,-79.490353,Bakery,York,43.6896,-79.4791
6,Mattachioni,43.664819,-79.454714,Bakery,York,43.6896,-79.4791
7,Pukka,43.681157,-79.429366,Indian Restaurant,York,43.6896,-79.4791
8,Propeller Coffee Co,43.659357,-79.444952,Café,York,43.6896,-79.4791
9,Contra Cafe,43.668930,-79.426097,Café,York,43.6896,-79.4791


### East Toronto <a name="id4.8"></a>

In [92]:
DToronto_data = TD[TD['Borough'] == 'East Toronto'].reset_index(drop=True)
DToronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West , Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar , The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923


In [93]:
address = 'East Toronto, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6814698, -79.3060214.


In [94]:
import requests

url = 'https://api.foursquare.com/v3/places/search?query=restaurant&ll=43.6814%2C-79.3060&radius=50000'

headers = {
    'Accept': 'application/json',
    'Authorization': 'fsq3yG2k5UVBywBORoDkq8cx2922ATAPkGqibnwmhcbZkWg='
            }

response = requests.get(url, headers=headers).json()

print(response)

{'results': [{'fsq_id': '58375c0214250b703b480d06', 'categories': [{'id': 13034, 'name': 'Café', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_', 'suffix': '.png'}}, {'id': 13065, 'name': 'Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_', 'suffix': '.png'}}], 'chains': [], 'distance': 802, 'geocodes': {'main': {'latitude': 43.678063, 'longitude': -79.31469}, 'roof': {'latitude': 43.678063, 'longitude': -79.31469}}, 'link': '/v3/places/58375c0214250b703b480d06', 'location': {'address': '1801 Gerrard St E', 'country': 'CA', 'cross_street': '', 'formatted_address': '1801 Gerrard St E, Toronto ON M4L 2B5', 'locality': 'Toronto', 'neighborhood': ['East Toronto'], 'postcode': 'M4L 2B5', 'region': 'ON'}, 'name': 'Bodega Henriette', 'related_places': {}, 'timezone': 'America/Toronto'}, {'fsq_id': '4ad9221cf964a520671821e3', 'categories': [{'id': 13067, 'name': 'African Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categor

In [95]:
venues_list = []
for record in response['results']:
    venues_list.append({
        'Neighborhood': 'name',
        'Neighborhood Latitude': 'lat',
        'Neighborhood Longitude': 'lng',
        'Venue': record['name'],
        'Venue Latitude': record['geocodes']['main']['latitude'],
        'Venue Longitude': record['geocodes']['main']['longitude'],
        'Venue Category': record['categories'][0]['name'],
        })

    # do stuff here to create your dataframe
    nearby_venues = pd.DataFrame(venues_list)
    print(nearby_venues)

  Neighborhood Neighborhood Latitude Neighborhood Longitude             Venue  \
0         name                   lat                    lng  Bodega Henriette   

   Venue Latitude  Venue Longitude Venue Category  
0       43.678063        -79.31469           Café  
  Neighborhood Neighborhood Latitude Neighborhood Longitude             Venue  \
0         name                   lat                    lng  Bodega Henriette   
1         name                   lat                    lng       Rendez-Vous   

   Venue Latitude  Venue Longitude      Venue Category  
0       43.678063       -79.314690                Café  
1       43.682651       -79.327389  African Restaurant  
  Neighborhood Neighborhood Latitude Neighborhood Longitude             Venue  \
0         name                   lat                    lng  Bodega Henriette   
1         name                   lat                    lng       Rendez-Vous   
2         name                   lat                    lng    The Beech Tr

In [96]:
def getNearbyVenues(names, latitudes, longitudes, radius=10000):
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        for record in response['results']:
            venues_list.append({
                'Neighborhood': 'name',
                'Neighborhood Latitude': 'lat',
                'Neighborhood Longitude': 'lng',
                'Venue': record['name'],
                'Venue Latitude': record['geocodes']['main']['latitude'],
                'Venue Longitude': record['geocodes']['main']['longitude'],
                'Venue Category': record['categories'][0]['name'],
                })

            # do stuff here to create your dataframe
            nearby_venues = pd.DataFrame(venues_list)
            print(nearby_venues)

In [97]:
nearby_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,name,lat,lng,Bodega Henriette,43.678063,-79.314690,Café
1,name,lat,lng,Rendez-Vous,43.682651,-79.327389,African Restaurant
2,name,lat,lng,The Beech Tree,43.680543,-79.288746,Gastropub
3,name,lat,lng,The Wren,43.682552,-79.327981,Bar
4,name,lat,lng,Tori's Bakeshop,43.672232,-79.290340,Bakery


In [98]:
ET= nearby_venues
ET.head()


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,name,lat,lng,Bodega Henriette,43.678063,-79.314690,Café
1,name,lat,lng,Rendez-Vous,43.682651,-79.327389,African Restaurant
2,name,lat,lng,The Beech Tree,43.680543,-79.288746,Gastropub
3,name,lat,lng,The Wren,43.682552,-79.327981,Bar
4,name,lat,lng,Tori's Bakeshop,43.672232,-79.290340,Bakery


In [99]:
ET['Nh']= 'East Toronto'
ET['Nh Lat']= '43.6814'
ET['Nh Long']= '-79.3060'
ET= ET.drop(['Neighborhood','Neighborhood Latitude','Neighborhood Longitude'],axis=1)
ET.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Nh,Nh Lat,Nh Long
0,Bodega Henriette,43.678063,-79.314690,Café,East Toronto,43.6814,-79.3060
1,Rendez-Vous,43.682651,-79.327389,African Restaurant,East Toronto,43.6814,-79.3060
2,The Beech Tree,43.680543,-79.288746,Gastropub,East Toronto,43.6814,-79.3060
3,The Wren,43.682552,-79.327981,Bar,East Toronto,43.6814,-79.3060
4,Tori's Bakeshop,43.672232,-79.290340,Bakery,East Toronto,43.6814,-79.3060


In [100]:
ET.rename(columns={'Nh':'Neighborhood'}, inplace=True)
ET.rename(columns={'Nh Lat':'Neighborhood Latitude'}, inplace=True)
ET.rename(columns={'Nh Long':'Neighborhood Longitude'}, inplace=True)
ET.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Neighborhood,Neighborhood Latitude,Neighborhood Longitude
0,Bodega Henriette,43.678063,-79.314690,Café,East Toronto,43.6814,-79.3060
1,Rendez-Vous,43.682651,-79.327389,African Restaurant,East Toronto,43.6814,-79.3060
2,The Beech Tree,43.680543,-79.288746,Gastropub,East Toronto,43.6814,-79.3060
3,The Wren,43.682552,-79.327981,Bar,East Toronto,43.6814,-79.3060
4,Tori's Bakeshop,43.672232,-79.290340,Bakery,East Toronto,43.6814,-79.3060


In [101]:
map_ET = folium.Map(location=[latitude, longitude], zoom_start=12)

# añadir marcadores al mapa
for lat, lng, borough in zip(ET['Venue Latitude'], ET['Venue Longitude'], ET['Venue']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ET)  
    
map_ET

### West Toronto <a name="id4.9"></a>

In [102]:
DToronto_data = TD[TD['Borough'] == 'West Toronto'].reset_index(drop=True)
DToronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M6H,West Toronto,"Dufferin , Dovercourt Village",43.669005,-79.442259
1,M6J,West Toronto,"Little Portugal , Trinity",43.647927,-79.419750
2,M6K,West Toronto,"Brockton , Parkdale Village , Exhibition Place",43.636847,-79.428191
3,M6P,West Toronto,"High Park , The Junction South",43.661608,-79.464763
4,M6R,West Toronto,"Parkdale , Roncesvalles",43.648960,-79.456325


In [103]:
address = 'West Toronto, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6671938, -79.4629678.


In [104]:
import requests

url = 'https://api.foursquare.com/v3/places/search?query=restaurant&ll=43.6671%2C-79.4629&radius=50000'

headers = {
    'Accept': 'application/json',
    'Authorization': 'fsq3yG2k5UVBywBORoDkq8cx2922ATAPkGqibnwmhcbZkWg='
            }

response = requests.get(url, headers=headers).json()

print(response)

{'results': [{'fsq_id': '54b2cb1f498e24216930af59', 'categories': [{'id': 13338, 'name': 'Seafood Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/seafood_', 'suffix': '.png'}}, {'id': 17077, 'name': 'Meat and Seafood Store', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/foodanddrink_', 'suffix': '.png'}}], 'chains': [], 'distance': 227, 'geocodes': {'main': {'latitude': 43.66539, 'longitude': -79.461494}, 'roof': {'latitude': 43.66539, 'longitude': -79.461494}}, 'link': '/v3/places/54b2cb1f498e24216930af59', 'location': {'address': '2766 Dundas St W', 'country': 'CA', 'cross_street': '', 'formatted_address': '2766 Dundas St W, Toronto ON M6P 1Y3', 'locality': 'Toronto', 'neighborhood': ['West Toronto'], 'postcode': 'M6P 1Y3', 'region': 'ON'}, 'name': 'Honest Weight', 'related_places': {}, 'timezone': 'America/Toronto'}, {'fsq_id': '540f2951498e7516839a7126', 'categories': [{'id': 13236, 'name': 'Italian Restaurant', 'icon': {'prefix': 'ht

In [105]:
venues_list = []
for record in response['results']:
    venues_list.append({
        'Neighborhood': 'name',
        'Neighborhood Latitude': 'lat',
        'Neighborhood Longitude': 'lng',
        'Venue': record['name'],
        'Venue Latitude': record['geocodes']['main']['latitude'],
        'Venue Longitude': record['geocodes']['main']['longitude'],
        'Venue Category': record['categories'][0]['name'],
        })

    # do stuff here to create your dataframe
    nearby_venues = pd.DataFrame(venues_list)
    print(nearby_venues)

  Neighborhood Neighborhood Latitude Neighborhood Longitude          Venue  \
0         name                   lat                    lng  Honest Weight   

   Venue Latitude  Venue Longitude      Venue Category  
0        43.66539       -79.461494  Seafood Restaurant  
  Neighborhood Neighborhood Latitude Neighborhood Longitude          Venue  \
0         name                   lat                    lng  Honest Weight   
1         name                   lat                    lng           Nodo   

   Venue Latitude  Venue Longitude      Venue Category  
0       43.665390       -79.461494  Seafood Restaurant  
1       43.665327       -79.465412  Italian Restaurant  
  Neighborhood Neighborhood Latitude Neighborhood Longitude  \
0         name                   lat                    lng   
1         name                   lat                    lng   
2         name                   lat                    lng   

                Venue  Venue Latitude  Venue Longitude      Venue Cate

In [106]:
def getNearbyVenues(names, latitudes, longitudes, radius=10000):
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        for record in response['results']:
            venues_list.append({
                'Neighborhood': 'name',
                'Neighborhood Latitude': 'lat',
                'Neighborhood Longitude': 'lng',
                'Venue': record['name'],
                'Venue Latitude': record['geocodes']['main']['latitude'],
                'Venue Longitude': record['geocodes']['main']['longitude'],
                'Venue Category': record['categories'][0]['name'],
                })

            # do stuff here to create your dataframe
            nearby_venues = pd.DataFrame(venues_list)
            print(nearby_venues)

In [107]:
nearby_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,name,lat,lng,Honest Weight,43.665390,-79.461494,Seafood Restaurant
1,name,lat,lng,Nodo,43.665327,-79.465412,Italian Restaurant
2,name,lat,lng,The Good Neighbour,43.662602,-79.471193,Café
3,name,lat,lng,Mattachioni,43.664819,-79.454714,Bakery
4,name,lat,lng,Tre Mari Bakery,43.676559,-79.448509,Bakery


In [108]:
WT= nearby_venues
WT.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,name,lat,lng,Honest Weight,43.665390,-79.461494,Seafood Restaurant
1,name,lat,lng,Nodo,43.665327,-79.465412,Italian Restaurant
2,name,lat,lng,The Good Neighbour,43.662602,-79.471193,Café
3,name,lat,lng,Mattachioni,43.664819,-79.454714,Bakery
4,name,lat,lng,Tre Mari Bakery,43.676559,-79.448509,Bakery


In [109]:
WT['Nh']= 'West Toronto'
WT['Nh Lat']= '43.6671'
WT['Nh Long']= '-79.4629'
WT= WT.drop(['Neighborhood','Neighborhood Latitude','Neighborhood Longitude'],axis=1)
WT.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Nh,Nh Lat,Nh Long
0,Honest Weight,43.665390,-79.461494,Seafood Restaurant,West Toronto,43.6671,-79.4629
1,Nodo,43.665327,-79.465412,Italian Restaurant,West Toronto,43.6671,-79.4629
2,The Good Neighbour,43.662602,-79.471193,Café,West Toronto,43.6671,-79.4629
3,Mattachioni,43.664819,-79.454714,Bakery,West Toronto,43.6671,-79.4629
4,Tre Mari Bakery,43.676559,-79.448509,Bakery,West Toronto,43.6671,-79.4629


In [110]:
WT.rename(columns={'Nh':'Neighborhood'}, inplace=True)
WT.rename(columns={'Nh Lat':'Neighborhood Latitude'}, inplace=True)
WT.rename(columns={'Nh Long':'Neighborhood Longitude'}, inplace=True)
WT.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Neighborhood,Neighborhood Latitude,Neighborhood Longitude
0,Honest Weight,43.665390,-79.461494,Seafood Restaurant,West Toronto,43.6671,-79.4629
1,Nodo,43.665327,-79.465412,Italian Restaurant,West Toronto,43.6671,-79.4629
2,The Good Neighbour,43.662602,-79.471193,Café,West Toronto,43.6671,-79.4629
3,Mattachioni,43.664819,-79.454714,Bakery,West Toronto,43.6671,-79.4629
4,Tre Mari Bakery,43.676559,-79.448509,Bakery,West Toronto,43.6671,-79.4629


In [111]:
map_WT = folium.Map(location=[latitude, longitude], zoom_start=12)

# añadir marcadores al mapa
for lat, lng, borough in zip(WT['Venue Latitude'], WT['Venue Longitude'], WT['Venue']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_WT)  
    
map_WT

In [150]:
WT

,Venue,Venue Latitude,Venue Longitude,Venue Category,Neighborhood,Neighborhood Latitude,Neighborhood Longitude
0,Honest Weight,43.665390,-79.461494,Seafood Restaurant,West Toronto,43.6671,-79.4629
1,Nodo,43.665327,-79.465412,Italian Restaurant,West Toronto,43.6671,-79.4629
2,The Good Neighbour,43.662602,-79.471193,Café,West Toronto,43.6671,-79.4629
3,Mattachioni,43.664819,-79.454714,Bakery,West Toronto,43.6671,-79.4629
4,Tre Mari Bakery,43.676559,-79.448509,Bakery,West Toronto,43.6671,-79.4629
5,Propeller Coffee Co,43.659357,-79.444952,Café,West Toronto,43.6671,-79.4629
6,Pizzeria Defina,43.648681,-79.449689,Pizzeria,West Toronto,43.6671,-79.4629
7,Brock Sandwich,43.658956,-79.440445,Fast Food Restaurant,West Toronto,43.6671,-79.4629
8,Home Baking Co,43.659042,-79.440038,Bakery,West Toronto,43.6671,-79.4629
9,Contra Cafe,43.668930,-79.426097,Café,West Toronto,43.6671,-79.4629


### Central Toronto <a name="id4.10"></a>

In [112]:
DToronto_data = TD[TD['Borough'] == 'Central Toronto'].reset_index(drop=True)
DToronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197
3,M5P,Central Toronto,Forest Hill North & West,43.696948,-79.411307
4,M4R,Central Toronto,North Toronto West,43.715383,-79.405678


In [113]:
address = 'Central Toronto, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6575014, -79.42194764414944.


In [114]:
import requests

url = 'https://api.foursquare.com/v3/places/search?query=restaurant&ll=43.6575%2C-79.4219&radius=50000'

headers = {
    'Accept': 'application/json',
    'Authorization': 'fsq3yG2k5UVBywBORoDkq8cx2922ATAPkGqibnwmhcbZkWg='
            }

response = requests.get(url, headers=headers).json()

print(response)

{'results': [{'fsq_id': '514a4961e4b004a36b8d9ea0', 'categories': [{'id': 13009, 'name': 'Cocktail Bar', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/cocktails_', 'suffix': '.png'}}, {'id': 13068, 'name': 'American Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_', 'suffix': '.png'}}, {'id': 13347, 'name': 'Tapas Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/tapas_', 'suffix': '.png'}}], 'chains': [], 'distance': 316, 'geocodes': {'main': {'latitude': 43.65462, 'longitude': -79.420627}, 'roof': {'latitude': 43.65462, 'longitude': -79.420627}}, 'link': '/v3/places/514a4961e4b004a36b8d9ea0', 'location': {'address': '797 College St', 'country': 'CA', 'cross_street': '', 'formatted_address': '797 College St, Toronto ON M6G 1C7', 'locality': 'Toronto', 'neighborhood': ['Downtown Toronto'], 'postcode': 'M6G 1C7', 'region': 'ON'}, 'name': 'Bar Isabel', 'related_places': {}, 'timezone': 'America/Toronto

In [115]:
venues_list = []
for record in response['results']:
    venues_list.append({
        'Neighborhood': 'name',
        'Neighborhood Latitude': 'lat',
        'Neighborhood Longitude': 'lng',
        'Venue': record['name'],
        'Venue Latitude': record['geocodes']['main']['latitude'],
        'Venue Longitude': record['geocodes']['main']['longitude'],
        'Venue Category': record['categories'][0]['name'],
        })

    # do stuff here to create your dataframe
    nearby_venues = pd.DataFrame(venues_list)
    print(nearby_venues)

  Neighborhood Neighborhood Latitude Neighborhood Longitude       Venue  \
0         name                   lat                    lng  Bar Isabel   

   Venue Latitude  Venue Longitude Venue Category  
0        43.65462       -79.420627   Cocktail Bar  
  Neighborhood Neighborhood Latitude Neighborhood Longitude             Venue  \
0         name                   lat                    lng        Bar Isabel   
1         name                   lat                    lng  La Bella Managua   

   Venue Latitude  Venue Longitude     Venue Category  
0       43.654620       -79.420627       Cocktail Bar  
1       43.662264       -79.425123  French Restaurant  
  Neighborhood Neighborhood Latitude Neighborhood Longitude             Venue  \
0         name                   lat                    lng        Bar Isabel   
1         name                   lat                    lng  La Bella Managua   
2         name                   lat                    lng    Pinky's Ca Phe   

   Venue

In [116]:
def getNearbyVenues(names, latitudes, longitudes, radius=10000):
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        for record in response['results']:
            venues_list.append({
                'Neighborhood': 'name',
                'Neighborhood Latitude': 'lat',
                'Neighborhood Longitude': 'lng',
                'Venue': record['name'],
                'Venue Latitude': record['geocodes']['main']['latitude'],
                'Venue Longitude': record['geocodes']['main']['longitude'],
                'Venue Category': record['categories'][0]['name'],
                })

            # do stuff here to create your dataframe
            nearby_venues = pd.DataFrame(venues_list)
            print(nearby_venues)

In [117]:
nearby_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,name,lat,lng,Bar Isabel,43.654620,-79.420627,Cocktail Bar
1,name,lat,lng,La Bella Managua,43.662264,-79.425123,French Restaurant
2,name,lat,lng,Pinky's Ca Phe,43.655569,-79.413765,Asian Restaurant
3,name,lat,lng,Bar Raval,43.655851,-79.409883,Lounge
4,name,lat,lng,Imanishi Japanese Kitchen,43.649644,-79.426516,Japanese Restaurant


In [118]:
CT= nearby_venues
CT.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,name,lat,lng,Bar Isabel,43.654620,-79.420627,Cocktail Bar
1,name,lat,lng,La Bella Managua,43.662264,-79.425123,French Restaurant
2,name,lat,lng,Pinky's Ca Phe,43.655569,-79.413765,Asian Restaurant
3,name,lat,lng,Bar Raval,43.655851,-79.409883,Lounge
4,name,lat,lng,Imanishi Japanese Kitchen,43.649644,-79.426516,Japanese Restaurant


In [119]:
CT['Nh']= 'Central Toronto'
CT['Nh Lat']= '43.6575'
CT['Nh Long']= '-79.4219'
CT= CT.drop(['Neighborhood','Neighborhood Latitude','Neighborhood Longitude'],axis=1)
CT.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Nh,Nh Lat,Nh Long
0,Bar Isabel,43.654620,-79.420627,Cocktail Bar,Central Toronto,43.6575,-79.4219
1,La Bella Managua,43.662264,-79.425123,French Restaurant,Central Toronto,43.6575,-79.4219
2,Pinky's Ca Phe,43.655569,-79.413765,Asian Restaurant,Central Toronto,43.6575,-79.4219
3,Bar Raval,43.655851,-79.409883,Lounge,Central Toronto,43.6575,-79.4219
4,Imanishi Japanese Kitchen,43.649644,-79.426516,Japanese Restaurant,Central Toronto,43.6575,-79.4219


In [120]:
CT.rename(columns={'Nh':'Neighborhood'}, inplace=True)
CT.rename(columns={'Nh Lat':'Neighborhood Latitude'}, inplace=True)
CT.rename(columns={'Nh Long':'Neighborhood Longitude'}, inplace=True)
CT.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Neighborhood,Neighborhood Latitude,Neighborhood Longitude
0,Bar Isabel,43.654620,-79.420627,Cocktail Bar,Central Toronto,43.6575,-79.4219
1,La Bella Managua,43.662264,-79.425123,French Restaurant,Central Toronto,43.6575,-79.4219
2,Pinky's Ca Phe,43.655569,-79.413765,Asian Restaurant,Central Toronto,43.6575,-79.4219
3,Bar Raval,43.655851,-79.409883,Lounge,Central Toronto,43.6575,-79.4219
4,Imanishi Japanese Kitchen,43.649644,-79.426516,Japanese Restaurant,Central Toronto,43.6575,-79.4219


In [121]:
map_CT = folium.Map(location=[latitude, longitude], zoom_start=12)

# añadir marcadores al mapa
for lat, lng, borough in zip(CT['Venue Latitude'], CT['Venue Longitude'], CT['Venue']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_CT)  
    
map_CT

In [152]:
CT

,Venue,Venue Latitude,Venue Longitude,Venue Category,Neighborhood,Neighborhood Latitude,Neighborhood Longitude
0,Bar Isabel,43.654620,-79.420627,Cocktail Bar,Central Toronto,43.6575,-79.4219
1,La Bella Managua,43.662264,-79.425123,French Restaurant,Central Toronto,43.6575,-79.4219
2,Pinky's Ca Phe,43.655569,-79.413765,Asian Restaurant,Central Toronto,43.6575,-79.4219
3,Bar Raval,43.655851,-79.409883,Lounge,Central Toronto,43.6575,-79.4219
4,Imanishi Japanese Kitchen,43.649644,-79.426516,Japanese Restaurant,Central Toronto,43.6575,-79.4219
5,Hanmoto,43.649508,-79.422988,Asian Restaurant,Central Toronto,43.6575,-79.4219
6,Pizzeria Libretto,43.648854,-79.420406,Pizzeria,Central Toronto,43.6575,-79.4219
7,Takht-e Tavoos,43.652521,-79.433241,Middle Eastern Restaurant,Central Toronto,43.6575,-79.4219
8,Foxley,43.648743,-79.420388,Bistro,Central Toronto,43.6575,-79.4219
9,Contra Cafe,43.668930,-79.426097,Café,Central Toronto,43.6575,-79.4219


## Analisis Final <a name="id5"></a>

A continuación crearemos una dataframe con la información extraída por cada vecindario.

In [122]:
finaldata= NY.append([DT,QP,ETO,SH,EY,YK,ET,WT,CT])
finaldata.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,Neighborhood,Neighborhood Latitude,Neighborhood Longitude
0,Grill Gate,43.753183,-79.451558,Fast Food Restaurant,North York,43.7543,-79.44911
1,Kinka Izakaya North York,43.760365,-79.410068,Japanese Restaurant,North York,43.7543,-79.44911
2,Bobbette & Belle,43.731388,-79.403655,Bakery,North York,43.7543,-79.44911
3,Sababa Restaurant,43.795348,-79.433724,Mediterranean Restaurant,North York,43.7543,-79.44911
4,Rustic Bakery,43.715504,-79.490353,Bakery,North York,43.7543,-79.44911


Ahora identificaremos el numero de restaurantes por vecindarios y el tipo de restaurante mas popular.

In [126]:
Neighborhood_counts = finaldata['Neighborhood'].value_counts().to_frame()
Neighborhood_counts.rename(columns={'Neighborhood': 'value_counts'}, inplace=True)
Neighborhood_counts

,value_counts
North York,10
Downtown Toronto,10
Queen's Park,10
Etobicoke,10
Scarborough,10
East York,10
York,10
East Toronto,10
West Toronto,10
Central Toronto,10


In [139]:
Venue_Category_counts = finaldata['Venue_Category'].value_counts().to_frame()
Venue_Category_counts.rename(columns={'Venue_Category': 'value_counts'}, inplace=True)
Venue_Category_counts

,value_counts
Bakery,16
Café,16
Bar,6
Steakhouse,4
Pizzeria,4
Japanese Restaurant,4
Italian Restaurant,4
Asian Restaurant,3
Restaurant,3
Fast Food Restaurant,2


Para realizar un análisis mas exacto, eliminaremos las panaderías y los cafés del dataframe

In [147]:
NoCoBa = finaldata.loc[(finaldata['Venue_Category'] != 'Bakery')&(finaldata['Venue_Category'] != 'Café')]
NoCoBa.head()

,Venue,Venue Latitude,Venue Longitude,Venue_Category,Neighborhood,Neighborhood Latitude,Neighborhood Longitude
0,Grill Gate,43.753183,-79.451558,Fast Food Restaurant,North York,43.7543,-79.44911
1,Kinka Izakaya North York,43.760365,-79.410068,Japanese Restaurant,North York,43.7543,-79.44911
3,Sababa Restaurant,43.795348,-79.433724,Mediterranean Restaurant,North York,43.7543,-79.44911
5,Eat BKK Thai Kitchen & Bar - North York,43.795793,-79.419115,Thai Restaurant,North York,43.7543,-79.44911
6,The Abbot on Eglinton,43.703719,-79.413372,Bar,North York,43.7543,-79.44911


In [148]:
Nhood_counts = NoCoBa['Neighborhood'].value_counts().to_frame()
Nhood_counts.rename(columns={'Neighborhood': 'value_counts'}, inplace=True)
Nhood_counts

,value_counts
Etobicoke,9
Central Toronto,9
Downtown Toronto,8
North York,7
Queen's Park,7
Scarborough,7
East York,7
East Toronto,6
York,4
West Toronto,4


In [149]:
VC_counts = NoCoBa['Venue_Category'].value_counts().to_frame()
VC_counts.rename(columns={'Venue_Category': 'value_counts'}, inplace=True)
VC_counts

,value_counts
Bar,6
Steakhouse,4
Pizzeria,4
Japanese Restaurant,4
Italian Restaurant,4
Asian Restaurant,3
Restaurant,3
Fast Food Restaurant,2
African Restaurant,2
Cocktail Bar,2


## Conclusiones <a name="id6"></a>

Después de realizar el análisis de todas las muestras podemos identificar que los vecindarios de York y de West Toronto son los que cuentan con menos restaurantes, pero con una cantidad considerable de cafés y panaderías siendo hasta un 60% de los restaurantes de la zona. El remanente se divide en distintos tipos de restaurantes, pero la gran mayoría es comida rápida (pizza y sándwiches).

Mientras que en Etobicoke, Central Toronto y Downtown Toronto son los vecindarios con mayor numero de restaurantes y solo el 10% de los negocios son cafés o panaderías. Además de una gran diversidad de tipos de restaurantes siendo la comida asiática y americana los más común en estos vecindarios.
